# Imatinib

In [1]:
%load_ext autoreload
%autoreload 2

### Libraries

In [2]:
import sys
sys.path.append("..")

In [3]:
import open3d as o3d
import numpy as np
import seaborn as sns
import pandas as pd

import re, os
from io import StringIO

from tqdm.auto import trange

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools

import ipywidgets as widgets

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-08-20 11:16:52,300 - __init__ - Enabling RDKit 2021.03.3 jupyter extensions


In [4]:
from utils import show_molecule_idx
from utils import AlignShow
from score_pcd import fit_and_score

## Create Point Clouds

Create PCD files for imatinib using Singularity container:

In [5]:
%%bash

singularity run --nv --app python ../development/densitymatch.sif \
    ../molgrid_to_pcd.py ../files/imatinib.sdf -m ../files/ligmap -r 0.5 -o ligands/imatinib.pcd
singularity run --nv --app python ../development/densitymatch.sif \
    ../molgrid_to_pcd.py ../files/imatinib_moved.sdf -m ../files/ligmap -r 0.5 -o ligands/imatinib_moved.pcd
singularity run --nv --app python ../development/densitymatch.sif \
    ../molgrid_to_pcd.py ../files/imatinib_part.sdf -m ../files/ligmap -r 0.5 -o ligands/imatinib_part.pcd

Using a Singularity container to run `molgrid_to_pcd.py` is needed because of a [`openbabel`/`molgrid` incomparibility from PyPI](https://github.com/gnina/libmolgrid/issues/62). `molgrid` and `openbabel` in the Singularity container are both installed form source.

## Imatinib Alignment

Load SDF and PCD files for imatibib (original coordinates and moved coordinates):

In [6]:
files = ["imatinib.pcd", "imatinib_moved.pcd"]

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(os.path.join("ligands", f))
    pcds.append(pcd)

    s = Chem.SDMolSupplier(os.path.join("../files",f.replace(".pcd", ".sdf")))
    mol = next(s)
    mols.append(mol)

In [7]:
print(pcds, mols)

[PointCloud with 1284 points., PointCloud with 1239 points.] [<rdkit.Chem.rdchem.Mol object at 0x7f8487763a80>, <rdkit.Chem.rdchem.Mol object at 0x7f8487763f30>]


We can visualize the molecules using `py3Dmol`:

In [8]:
_ = widgets.interact(lambda index: show_molecule_idx(index, mols), index=widgets.IntSlider(min=0, max=len(mols)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='index', max=1), Output()), _dom_classes=('widget-interac…

We can now fit the imatinib (as point cloud) into imatinib and score the alignmenment:

In [9]:
gfit, cfit, tran = fit_and_score(pcds, voxel_size=0.5, threshold=0.5)

In [10]:
print("GFIT:", gfit.fitness)
print("CFIT:", cfit.fitness)
print("transformation:\n", tran)

GFIT: 0.8886292834890965
CFIT: 0.9587227414330218
transformation:
 [[ -0.22349726  -0.96928013  -0.10268889   4.2042644 ]
 [ -0.72704401   0.09561122   0.67990109   0.47138654]
 [ -0.64919641   0.22661538  -0.72607816 -10.65679689]
 [  0.           0.           0.           1.        ]]


The alignment also returns a $4\times 4$ representing the affine transformation (rotation and translation) needed to superimpose the two point clouds (and molecules). The affine transformation is defined by the following $4\times 4$ matrix:
$$
    A = \begin{pmatrix}
        r_{00} & r_{01} & r_{02} & t_0 \\
        r_{10} & r_{11} & r_{12} & t_1 \\
        r_{20} & r_{21} & r_{22} & t_2 \\
        0 & 0 & 0 & 1 \\
    \end{pmatrix}
$$
For an affine tranfromation $A$ represented in this way, the transformed points $\mathbf{x}'$ are given by:
$$
    \begin{pmatrix}
        x' \\ y' \\ z' \\ 1
    \end{pmatrix} = 
    A \begin{pmatrix}
        x \\ y \\ z \\ 1
    \end{pmatrix}
$$

We can now transform the corrdinates of the RDKit molecules in order to check the superimposition:

In [11]:
# Molecule to transform
molid = 0

In [12]:

# Select coordinates to transform
coords = mols[molid].GetConformer(0).GetPositions()

# Augment coordinates with ones
coords_aug = np.ones((coords.shape[0], 4))
coords_aug[:,:3] = coords

# Transform coordinates (using broadcasting)
coords_new = np.matmul(tran, coords_aug.T)[:3,:].T

In [13]:
# Add new coordinates as additional conformer
n_atoms = mols[molid].GetNumAtoms()
conf = Chem.Conformer(n_atoms)
for i in range(n_atoms):
    conf.SetAtomPosition(i, coords_new[i,:])

_ = mols[molid].AddConformer(conf, assignId=True)

All this functionality is included in `AlignShow`, which also allow post-alignment visualization:

In [14]:
als = AlignShow(mols, pcds)

We can now visually check the alignment:

In [15]:
als.show(0, 1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
als.show(1, 0)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Imatininib Alignment with Subset

We can now try to align a subset (a small fragment) of imatinib into the whole molecule:

In [17]:
files = ["imatinib.pcd", "imatinib_part.pcd"]

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(os.path.join("ligands", f))
    pcds.append(pcd)

    s = Chem.SDMolSupplier(os.path.join("../files",f.replace(".pcd", ".sdf")))
    mol = next(s)
    mols.append(mol)

In [18]:
print(pcds, mols)

[PointCloud with 1284 points., PointCloud with 477 points.] [<rdkit.Chem.rdchem.Mol object at 0x7f8487763ee0>, <rdkit.Chem.rdchem.Mol object at 0x7f848762a350>]


In [19]:
als = AlignShow(mols, pcds)
als.show(1, 0)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Unfortunately the point cloud alignment is non-deterministic (it is impossible to set a random seed, see [open3d/#288](https://github.com/intel-isl/Open3D/issues/288) and [open3d/#1263](https://github.com/intel-isl/Open3D/issues/1263)) and therefore sometimes the alignment is extremely good while other times it is not.